In [1]:
import json
import pystac
import stackstac
import os
import xarray as xr
import geopandas as gpd
from shapely import Polygon
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray as rio
from rasterio.crs import CRS
import rasterio 
import matplotlib.patches as mpatches
import numpy as np
import scipy
from scipy.stats import sem

In [2]:
import sys
sys.path.insert(0, '/uufs/chpc.utah.edu/common/home/u1269862/2023/new_retreat/retreat/')

import retreat_tools
import itslive_tools
import general_tools

## RETREAT data

In [3]:
catalog = pystac.Catalog.from_file('/uufs/chpc.utah.edu/common/home/u1269862/2023/new_retreat/324stac_catalog/catalog.json')


In [4]:
items = list(catalog.get_all_items())

In [5]:
for item in items:
    
    retreat_tools.check_orig_files(item)

In [6]:
cube = stackstac.stack(
    items = [item.to_dict() for item in items])

In [7]:
cube = retreat_tools.cube_process(cube)

In [8]:
cube

<xarray.DataArray 'stackstac-54a52d7516c7c79813764bc90c09e972' (time: 314,
                                                                band: 5,
                                                                y: 1951, x: 1875)>
dask.array<fetch_raster_window, shape=(314, 5, 1951, 1875), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/18)
  * time            (time) datetime64[ns] 2014-10-23 2014-10-31 ... 2021-11-05
    id              (time) <U19 '10_085_002_20141023' ... '10_019_085_20211105'
  * band            (band) <U9 'dis_ang' 'dis_az' 'dis_mag' 'dis_N_ang' 'dis_r'
  * x               (x) float64 1.128e+05 1.13e+05 ... 4.874e+05 4.876e+05
  * y               (y) float64 3.357e+06 3.357e+06 ... 2.967e+06 2.967e+06
    sensor          <U2 'S1'
    ...              ...
    title           (band) <U9 'dis_ang' 'dis_az' 'dis_mag' 'dis_N_ang' 'dis_r'
    proj:epsg       int64 32645
    epsg            int64 32645
    img1_date       (time) datetime64[ns] 2014-10-11T12:21:55 ... 2021-10-30T...
    img2_date       (time) datetime64[ns] 2014-11-04T12:21:55 ... 2021-11-11T...
    img_separation  (time) float64 -24.0 -24.0 -24.0 -24.0 ... -12.0 -12.0 -12.0
Attributes:
    spec:        RasterSpec(epsg=32645, bounds=(112800.0, 2966800.0, 487800.0...
    crs:         epsg:32645
    transform:   | 200.00, 0.00, 112800.00|\n| 0.00,-200.00, 3357000.00|\n| 0...
    resolution:  200.0

## Aux data

### RGI

In [9]:
rgi_path = '/uufs/chpc.utah.edu/common/home/u1269862/2023/new_retreat/data/rgi/'

In [10]:
rgi15 = gpd.read_file(os.path.join(rgi_path, 'rgi15/15_rgi60_SouthAsiaEast.shp'))

In [11]:
rgi15_prj = rgi15.to_crs('EPSG:32645')

In [12]:
rgi_ids = pd.read_csv('/uufs/chpc.utah.edu/common/home/u1269862/2023/new_retreat/data/manuscript_rgi_ids.csv', )

lake_ids = ['RGI60-15.10255', 'RGI60-15.10285', 'RGI60-15.10279','RGI60-15.10286',
            'RGI60-15.09361','RGI60-15.09483','RGI60-15.10290','RGI60-15.10299']

In [13]:
rgi_subset = rgi15_prj.loc[rgi15_prj['RGIId'].isin(rgi_ids['RGI_IDs'].to_list())]

In [14]:
rgi_ids = rgi_subset['RGIId'].to_list()

In [15]:
rgi_lakes = rgi_subset.loc[rgi_subset['RGIId'].isin(lake_ids)]
rgi_lands = rgi_subset.loc[~rgi_subset['RGIId'].isin(lake_ids)]

In [16]:
land_ids = rgi_lands['RGIId'].to_list()

### NASADEM

In [17]:
nasadem_dir = '/uufs/chpc.utah.edu/common/home/u1269862/2023/new_retreat/data/nasadem/'

In [18]:
nasadem = xr.open_dataset(os.path.join(nasadem_dir,  'NASADEM_NC_n28e085.nc'))

In [19]:
nasadem = nasadem.rio.write_crs('EPSG:4326')

In [20]:
nasadem_prj = nasadem.rio.reproject('EPSG:32645')

## Make dicts of clipped glacier velocities

In [21]:
ds_10255 = retreat_tools.clip_glacier_add_dem('RGI60-15.10255',rgi_subset, cube, nasadem_prj, 'full')

retreat clipped
cov done
dem clipped
sem calculated
z stuff
finished z stuff
elevation masking should be all good


In [23]:
ds_10255.to_netcdf('/uufs/chpc.utah.edu/common/home/cryosphere/emarshall/44_retreat_clipped/rgi10255_ds.nc','w', engine='netcdf4', format='NETCDF4')

In [22]:
ds_10255

<xarray.Dataset>
Dimensions:         (time: 158, x: 34, y: 43)
Coordinates: (12/18)
  * time            (time) datetime64[ns] 2014-12-22 2014-12-30 ... 2021-04-27
  * x               (x) float64 3.738e+05 3.74e+05 ... 3.802e+05 3.804e+05
  * y               (y) float64 3.146e+06 3.146e+06 ... 3.138e+06 3.138e+06
    id              (time) <U19 '10_085_002_20141222' ... '10_019_085_20210427'
    sensor          <U2 'S1'
    img2_id         (time) <U4 'CC73' '5BFB' '1978' ... '3591' '520F' 'B50F'
    ...              ...
    epsg            int64 32645
    img1_date       (time) datetime64[ns] 2014-11-28T12:21:54 ... 2021-04-21T...
    img2_date       (time) datetime64[ns] 2015-01-15T12:21:53 ... 2021-05-03T...
    img_separation  (time) float64 -48.0 -48.0 -24.0 -24.0 ... -13.0 -12.0 -13.0
    spatial_ref     int64 0
    crs             int64 0
Data variables: (12/17)
    dis_ang         (time, y, x) float64 dask.array<chunksize=(1, 43, 34), meta=np.ndarray>
    dis_az          (time, y, x) float64 dask.array<chunksize=(1, 43, 34), meta=np.ndarray>
    dis_mag         (time, y, x) float64 dask.array<chunksize=(1, 43, 34), meta=np.ndarray>
    dis_N_ang       (time, y, x) float64 dask.array<chunksize=(1, 43, 34), meta=np.ndarray>
    dis_r           (time, y, x) float64 dask.array<chunksize=(1, 43, 34), meta=np.ndarray>
    cov             (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...              ...
    z2              (y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    z3              (y, x) float32 nan nan nan nan nan ... 7.284e+03 nan nan nan
    z0_sem          (time) float64 1.193 0.8928 0.7689 ... 1.404 1.696 1.721
    z1_sem          (time) float64 2.051 2.551 5.785 3.995 ... 4.399 3.145 5.949
    z2_sem          (time) float64 2.184 1.839 3.282 3.159 ... 2.728 2.69 3.06
    z3_sem          (time) float64 1.722 1.846 2.257 1.65 ... 1.894 2.173 1.855
Attributes:
    spec:        RasterSpec(epsg=32645, bounds=(112800.0, 2966800.0, 487800.0...
    resolution:  200.0

In [21]:
#lake_ds_ls = [] 
#for glacier in range(len(lake_ids)):
    
#    ds = retreat_tools.clip_glacier_add_dem(lake_ids[glacier], rgi_lakes, cube, nasadem_prj, 'full')
#    ds.to_netcdf(f'/uufs/chpc.utah.edu/common/home/u1269862/2023/new_retreat/43_results/retreat/ds_{lake_ids[glacier]}.nc')
#lakes_dict = dict(zip(lake_ids, lake_ds_ls))

In [ ]:
rgi_subset

In [ ]:
already_made_dfs = os.listdir('/uufs/chpc.utah.edu/common/home/u1269862/2023/new_retreat/43_results/retreat/')

already_made_dfs = [glacier[3:-4] for glacier in already_made_dfs]

ds_ls = [] 

for element in range(len(rgi_ids)):
    
    rgi_id = rgi_ids[element]
    print(rgi_id)
    
    if rgi_id in already_made_dfs:
        
        print('already done')
    else:
        try:
            ds = retreat_tools.clip_glacier_add_dem(rgi_ids[element], rgi_subset, cube, nasadem_prj, 'full')
            ds.to_netcdf(f'/uufs/chpc.utah.edu/common/home/cryosphere/emarshall/44_retreat_clipped/ds_{land_ids[glacier]}.nc')
        except:
            print('probably an empty time step in this glacier?')
rgi_dict = dict(zip(rgi_ids, ds_ls))




RGI60-15.03976
retreat clipped
cov done
dem clipped
sem calculated
z stuff
finished z stuff
elevation masking should be all good
probably an empty time step in this glacier?
RGI60-15.04119
retreat clipped
cov done
dem clipped
sem calculated
z stuff
finished z stuff
elevation masking should be all good
probably an empty time step in this glacier?
RGI60-15.04121
retreat clipped
cov done
dem clipped
sem calculated
z stuff
finished z stuff
elevation masking should be all good
probably an empty time step in this glacier?
RGI60-15.04176
retreat clipped
cov done
dem clipped
sem calculated
z stuff
finished z stuff
elevation masking should be all good
probably an empty time step in this glacier?
RGI60-15.04220
retreat clipped
cov done
dem clipped
sem calculated
z stuff
finished z stuff
elevation masking should be all good
probably an empty time step in this glacier?
RGI60-15.09249
retreat clipped
cov done
dem clipped
sem calculated
z stuff
finished z stuff
elevation masking should be all good
p